### Cross-functional with one data scientisg working on an ML model

- tracking server: yes, local sever
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing local tracking server.
To run this example you need to luanch the mlflow server locally by running following command in your terminal:
``` 
mlflow server --backend-store-uri sqlite:///backend.db
```

In [1]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [4]:
mlflow.list_experiments()

[<Experiment: artifact_location='./artifacts_local/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>]

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

# Set up experiment
mlflow.set_experiment("my-experiment-1")

2022/08/06 19:50:26 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


<Experiment: artifact_location='./artifacts_local/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>

In [7]:
with mlflow.start_run():
    X, y = load_iris(return_X_y= True)

    params = {"C" : 0.05, "random_state" : 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)

    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path = "models")
    print(f"defaul artifact URI : {mlflow.get_artifact_uri()}")

defaul artifact URI : ./artifacts_local/1/40844abebf3f49a78fd3ef35dff14fbf/artifacts


In [10]:
mlflow.list_experiments()

[<Experiment: artifact_location='./artifacts_local/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./artifacts_local/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

In [8]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

In [9]:
client.list_registered_models()

[]

In [12]:
run_id = client.list_run_infos(experiment_id = '1')[0].run_id
mlflow.register_model(
    model_uri = f"runs:/{run_id}/models",
    name = 'iris-simple-classifier'
)

Successfully registered model 'iris-simple-classifier'.
2022/08/06 19:56:28 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-simple-classifier, version 1
Created version '1' of model 'iris-simple-classifier'.


<ModelVersion: creation_timestamp=1659790588620, current_stage='None', description='', last_updated_timestamp=1659790588620, name='iris-simple-classifier', run_id='40844abebf3f49a78fd3ef35dff14fbf', run_link='', source='./artifacts_local/1/40844abebf3f49a78fd3ef35dff14fbf/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [15]:
# Another models 
run_id = client.list_run_infos(experiment_id = '1')[-1].run_id
mlflow.register_model(
    model_uri = f"runs:/{run_id}/models",
    name = 'iris-simple-classifier'
)

Registered model 'iris-simple-classifier' already exists. Creating a new version of this model...
2022/08/06 20:08:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-simple-classifier, version 2
Created version '2' of model 'iris-simple-classifier'.


<ModelVersion: creation_timestamp=1659791307258, current_stage='None', description='', last_updated_timestamp=1659791307258, name='iris-simple-classifier', run_id='98f05dbde5bf4c0293fb0ce88f63f605', run_link='', source='./artifacts_local/1/98f05dbde5bf4c0293fb0ce88f63f605/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='2'>

In [13]:
client.list_registered_models() 

[<RegisteredModel: creation_timestamp=1659790588592, description='', last_updated_timestamp=1659790588620, latest_versions=[<ModelVersion: creation_timestamp=1659790588620, current_stage='None', description='', last_updated_timestamp=1659790588620, name='iris-simple-classifier', run_id='40844abebf3f49a78fd3ef35dff14fbf', run_link='', source='./artifacts_local/1/40844abebf3f49a78fd3ef35dff14fbf/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-simple-classifier', tags={}>]